# **Neural Networks - Deep Learning**
## Exercise 2 - Classification using SVM Models

### Georgios Tsoumplekas, AEM: 9359

# **Part 0:** Laying the ground

## **Imports**

In [ ]:
import tensorflow as tf

from tensorflow.keras.datasets import mnist
from keras.utils import np_utils

from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report

import numpy as np
from numpy import linalg

import time
import random as rnd

In [ ]:
!pip install cvxopt

In [ ]:
from cvxopt import matrix, solvers

## **Preprocesing of the MNIST dataset**

In [ ]:
#MNIST dataset parameters
num_classes = 2 #total classes: odd and even numbers
num_features = 784 #data features (28x28 pixels image)

In [ ]:
#Load data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print("x_train shape", x_train.shape) 
print("y_train shape", y_train.shape)
print("x_test shape", x_test.shape)
print("y_test shape", y_test.shape)

#Convert to float32
x_train, x_test = np.array(x_train, dtype=np.float32), np.array(x_test, dtype=np.float32)
y_train, y_test = np.array(y_train, dtype=np.float32), np.array(y_test, dtype=np.float32)

#Flatten image to 1-D vector
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])

#Normalize to [0,1]
x_train, x_test = x_train/255.0, x_test/255.0

# Change labels of training set: -1 for even numbers, 1 for odd numbers
even = np.where(y_train%2 == 0)
odd = np.where(y_train%2 == 1)
y_train[even] = -1
y_train[odd] = 1

print("y_ train even shape:", y_train[even].shape)
print("y_ train odd shape:", y_train[odd].shape)

# Change labels of test set: -1 for even numbers, 1 for odd numbers
even = np.where(y_test%2 == 0)
odd = np.where(y_test%2 == 1)
y_test[even] = -1
y_test[odd] = 1

print("y_ test even shape:", y_test[even].shape)
print("y_ test odd shape:", y_test[odd].shape)

11501568/11490434 [==============================] - 0s 0us/step
x_train shape (60000, 28, 28)
y_train shape (60000,)
x_test shape (10000, 28, 28)
y_test shape (10000,)
y_ train even shape: (29492,)
y_ train odd shape: (30508,)
y_ test even shape: (4926,)
y_ test odd shape: (5074,)


### **Dimensionality reduction using PCA**

In [ ]:
start = time.time()

pca = PCA(n_components=0.9).fit(x_train)
x_train_pca = pca.transform(x_train)
x_test_pca = pca.transform(x_test)

end = time.time()
print("PCA elapsed time: {}s\n".format(end-start))
print("We extract {} feautures from the original {}.".format(x_train_pca.shape[1],x_train.shape[1]))
print('Cumulative explained variation for {} principal components: {}'.format(x_train_pca.shape[1], np.sum(pca.explained_variance_ratio_)))

PCA elapsed time: 7.42681097984314s

We extract 87 feautures from the original 784.
Cumulative explained variation for 87 principal components: 0.9001063108444214


Our goal was that the new components (features) achieve 90% explained variance of the initial ones. We see that by creating only 87 of them we can achieve this number (roughly 1/10th of the initial 784 features). 

# **Part 1:** SVM implemented with QP solver

An SVM works by dealing with the classification problem as an optimization problem. More specifically, the training process of an SVM consists of finding the optimal hyperplane that maximizes the margin between this hyperplane and the samples of the 2 classes that lie closer to it (the support vectors). \
 This is a well known optimization problem that can be analyzed using Lagrange multipliers and be solved using Quadratic Programming. We can adjust the problem's parameters so that we can allow some of the samples to be misclassified (by adding slack variables) or transform the data to a higher-dimensionality space using kernel functions hoping that they become linearly separable. \
 

## Define kernel functions

The linear kernel works pretty well when the data are (or almost are) linearly separable.

In [ ]:
def linear_kernel(x1, x2):
    return np.dot(x1, x2.T)

The gaussian kernel can help by transforming the data to a higher-dimensionality space where we hope that the data are linearly separable. Here, we don't actually transform the data to the high-dimensional feature space but instead use the 'kernel trick' to do the necessary calculations in the initial low-dimensional feature space.

In [ ]:
def gaussian_kernel(x, y, gamma):
    return np.exp(-gamma*linalg.norm(x-y)**2)

## Define SVM model

In [ ]:
class SVM(object):

    def __init__(self, kernel=linear_kernel, C=None, gamma=None, disp=True):
        self.kernel = kernel
        self.C = C
        self.gamma = gamma
        self.disp = disp
        if self.C is not None: 
            self.C = float(self.C)
    
    def fit(self, x_train, y_train):        
        n_samples, n_features = x_train.shape

        if self.disp is True:
            print('Number of samples:', n_samples)
            print('Number of features:', n_features)

        # Calculate Gram matrix
        start = time.time()
        K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                if self.kernel is linear_kernel:
                    K[i,j] = self.kernel(x_train[i], x_train[j])
                else:
                    K[i,j] = self.kernel(x_train[i], x_train[j],self.gamma)
        end = time.time()
        if self.disp is True:
            print("Gram matrix elapsed time: {}s\n".format(end-start))
        
        # Formulate the matrices of the QP problem
        P = matrix(np.outer(y_train,y_train) * K)
        q = matrix(np.ones(n_samples) * -1.0)
        A = matrix(y_train.astype('double'), (1,n_samples))
        b = matrix(0.0)
        
        if self.C is None:
            G = matrix(np.diag(np.ones(n_samples) * -1))
            h = matrix(np.zeros(n_samples))
        else:
            G_1 = -1*np.identity(n_samples)
            G_2 = np.identity(n_samples)
            G = matrix(np.vstack((G_1, G_2)))
            h_1 = np.zeros(n_samples)
            h_2 = np.ones(n_samples) * self.C
            h = matrix(np.hstack((h_1, h_2)))
          
        # Solve QP problem
        start = time.time()
        solvers.options['show_progress'] = False
        solution = solvers.qp(P, q, G, h, A, b, )
        end = time.time()

        if self.disp is True:
            print("QP solver elapsed time: {}s\n".format(end-start))
        
        # Lagrange multipliers
        a = np.ravel(solution['x'])
        
        # Find Support Vectors
        sv = a > 1e-3
        self.a = a[sv]
        self.sv = x_train[sv]
        self.sv_y = y_train[sv]
        ind = np.arange(len(a))[sv] #just a trick to get the indexes which are also stored in sv
        
        # Bias (calculated for each sv and then we take the mean for numerical stability reasons)
        self.b = 0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * K[ind[n],sv])
        self.b /= len(self.a)
        
    def predict(self, x_test):
        start = time.time()
        y_pred = np.zeros(len(x_test))
        for i in range(len(x_test)):
            for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                if self.kernel is linear_kernel:
                    y_pred[i] += a * sv_y * self.kernel(x_test[i], sv)
                else:
                    y_pred[i] += a * sv_y * self.kernel(x_test[i], sv, self.gamma)
        y_pred += self.b
        end = time.time()
        if self.disp is True:
            print("Testing elapsed time: {}s\n".format(end-start))
        return np.sign(y_pred).astype(int)

One of the main problems of solving the optimization problem using  Quadratic Programming (QP) is that the formulation of the problem requires the knowledge of the whole Gram matrix. However, the Gram matrix can become extremely big when we have to deal with a big training set rendering it unable to fit in the memory. In our example, to demonstrate the SVM model that uses a QP solver and test its performance afterwards, we will train the model in a subset of the original training set.

### Linear SVM with C=1

In [ ]:
# The training subset consists of 2000 samples
rand_indx = np.random.randint(0,len(x_train_pca),2000)
x_train_mini = x_train_pca[rand_indx,:]
y_train_mini = y_train[rand_indx]

# Train the model
svm = SVM(kernel=linear_kernel, C=1, gamma=None, disp=True)
svm.fit(x_train_mini,y_train_mini)

# Evaluate on test set
y_pred = svm.predict(x_test_pca)

print('Test set accuracy: ', accuracy_score(y_test,y_pred))
print(classification_report(y_test, y_pred))

Number of samples: 2000
Number of features: 87
Gram matrix elapsed time: 10.590656280517578s

QP solver elapsed time: 12.363476991653442s

Testing elapsed time: 21.558725833892822s

Test set accuracy:  0.8726
              precision    recall  f1-score   support

        -1.0       0.88      0.86      0.87      4926
         1.0       0.87      0.89      0.88      5074

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000



As we can see, half of the training time was used to calculate the Gram matrix and the other half for solving the optimization problem. Moreover, the model has no problem making predictions over a large test set, but it is clear that inference here is not as fast as in an MLP model per se. As for its performance, we can see that it is pretty good but not excelent. 

### SVM with RBF kernel: C=1, gamma=0.01

In [ ]:
svm = SVM(kernel=gaussian_kernel, C=1, gamma=1e-2, disp=True)
svm.fit(x_train_mini,y_train_mini)
y_pred = svm.predict(x_test_pca)

print('Test set accuracy: ', accuracy_score(y_test,y_pred))
print(classification_report(y_test, y_pred))

Number of samples: 2000
Number of features: 87
Gram matrix elapsed time: 60.9993999004364s

QP solver elapsed time: 8.565208911895752s

Testing elapsed time: 121.89988827705383s

Test set accuracy:  0.9495
              precision    recall  f1-score   support

        -1.0       0.95      0.95      0.95      4926
         1.0       0.95      0.95      0.95      5074

    accuracy                           0.95     10000
   macro avg       0.95      0.95      0.95     10000
weighted avg       0.95      0.95      0.95     10000



We can see that calculation of the Gram matrix is 6 times slower for the same training set compared to the linear SVM. However, the optimization problem takes less time to be solved. This is probably due to the fact that the transformed data are probably more linearly separable making the optimization problem easier to solve. Additionaly, the time needed for inference on the test set is also 6 times bigger than before, due to the fact that the test samples have to also pass through the RBF kernel. \
Regarding the performance of the model, we can see that it performs clearly better in comparison with the linear SVM. This leads us to understand that the initial data are not linearly separable but the kernel transformation makes them more linearly separable in the high-dimensional space.

# **Part 2:** Hyper-parameter tuning

An important aspect of creating an appropriate model for a specific dataset, is to define the optimal hyper-parameters for it. \
In order to find the optimal model we will apply grid search. Each of the candidate models will then be cross-validated using stratified 5-fold cross validation in order to achieve more robust estimations of the accuracy of each model. \
Finaly, the model with the best accuracy is being chosen.

### Fine-tuning the linear SVM

In the case of the linear SVM the only hyperparameter that needs to be defined is C. C controls the comprimise we make between the complexity of the model and the number of wrongly classified training samples. High values of parameter C are used when we do not want to allow many misclassified samples while smaller values of C are used when we know that the samples are not linearly separable and it's better to allow more misclassified samples but have a model with greater generalization power.

In [ ]:
def find_best_linear_SVM(k,c_vals,x_train,y_train):

    kfold = StratifiedKFold(n_splits=k,shuffle=True)

    acc_per_fold = np.zeros(k)
    best_acc = -1.0
    best_C = None

    model_no = 1

    for i in c_vals:
        fold_no = 0

        # Create model
        svm_model = SVM(kernel=linear_kernel,C=i,disp=False)

        for train, test in kfold.split(x_train, y_train):
            # Train model
            svm_model.fit(x_train[train],y_train[train])

            # Evaluate model on validation set
            y_pred = svm_model.predict(x_train[test])
            acc_per_fold[fold_no] = accuracy_score(y_train[test],y_pred)

            fold_no += 1

        # Mean of accuracy for 5-fold cross-validation of a specific model
        acc = np.mean(acc_per_fold)

        print('Model {}:'.format(model_no))
        print('C = ', i)
        print('Accuracy: ', acc)
        print('\n')

        model_no += 1

        # Choose best model based on its accuracy
        if(acc > best_acc):
            best_acc = acc
            best_C = i

    print('Optimal parameters for the model:')
    print('C = ', best_C)
    print('Accuracy: ', best_acc)

    return best_C

In [ ]:
# Find best C for the linear SVM
k = 5
c_vals = np.array([1, 10, 1e2, 1e3, 1e4])
best_C = find_best_linear_SVM(k,c_vals,x_train_mini,y_train_mini)

Model 1:
C =  1.0
Accuracy:  0.8605


Model 2:
C =  10.0
Accuracy:  0.8614999999999998


Model 3:
C =  100.0
Accuracy:  0.8645000000000002


Model 4:
C =  1000.0
Accuracy:  0.858


Model 5:
C =  10000.0
Accuracy:  0.8539999999999999


Optimal parameters for the model:
C =  100.0
Accuracy:  0.8645000000000002


We can see that the difference between the accuracy of the different candidate models is pretty small meaning that the performance of the linear SVM is not heavily affected by the choice of C in this particular problem.

In [ ]:
# Define optimal linear SVM
svm_model = SVM(kernel=linear_kernel, C=best_C, disp=True)

# Train model
svm_model.fit(x_train_mini,y_train_mini)

# Evaluate model on test set
y_pred = svm_model.predict(x_test_pca)

print('Test set accuracy: ', accuracy_score(y_test,y_pred))
print(classification_report(y_test, y_pred))

Number of samples: 2000
Number of features: 87
Gram matrix elapsed time: 10.598548412322998s

QP solver elapsed time: 17.644381284713745s

Testing elapsed time: 21.190078496932983s

Test set accuracy:  0.8738
              precision    recall  f1-score   support

        -1.0       0.88      0.86      0.87      4926
         1.0       0.87      0.89      0.88      5074

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000



We can verify that the choice of C doesn't seem to play a really important role here from the fact that the optimal model performs only slightly better than the linear SVM that we had in Part 1.

### Fine-tuning the Gaussian-kernel SVM

In the case of the Gaussian-kernel SVM, apart from C we have to also define hyperparameter gamma. Gamma affects the shape of the Gaussian kernel, with bigger values making the shape of the gaussian function more "flat" and smaller values of gamma making it more "steep".

In [ ]:
def find_best_gaussian_SVM(k,c_vals,gamma_vals,x_train,y_train):
    kfold = StratifiedKFold(n_splits=k,shuffle=True)

    acc_per_fold = np.zeros(k)
    best_acc = -1.0
    best_C = None
    best_gamma = None

    model_no = 1
    
    for i in c_vals:
        for j in gamma_vals:
            
            fold_no = 0

            # Create model
            svm_model = SVM(kernel=gaussian_kernel, C=i, gamma=j ,disp=False)

            for train, test in kfold.split(x_train, y_train):
                # Train model
                svm_model.fit(x_train[train],y_train[train])

                # Evaluate model on validation set
                y_pred = svm_model.predict(x_train[test])
                acc_per_fold[fold_no] = accuracy_score(y_train[test],y_pred)

                fold_no += 1
                
            # Mean of accuracy for 5-fold cross-validation of a specific model
            acc = np.mean(acc_per_fold)

            print('Model {}:'.format(model_no))
            print('C = ', i)
            print('gamma = ', j)
            print('Accuracy: ', acc)
            print('\n')

            model_no += 1

            # Choose best model based on its accuracy
            if(acc > best_acc):
                best_acc = acc
                best_C = i
                best_gamma = j
                
    print('\nOptimal parameters for the model:')
    print('C = ', best_C)
    print('gamma = ',best_gamma)
    print('Accuracy: ', best_acc)

    return best_C, best_gamma

In [ ]:
# Find best C and gamma for the gaussian-kernel SVM
k = 5
c_vals = np.array([1, 1e2, 1e4])
gamma_vals = np.array([1e-3, 1e-2, 1e-1])
best_C, best_gamma = find_best_gaussian_SVM(k,c_vals,gamma_vals,x_train_mini,y_train_mini)

Model 1:
C =  1.0
gamma =  0.001
Accuracy:  0.8685


Model 2:
C =  1.0
gamma =  0.01
Accuracy:  0.945


Model 3:
C =  1.0
gamma =  0.1
Accuracy:  0.916


Model 4:
C =  100.0
gamma =  0.001
Accuracy:  0.9309999999999998


Model 5:
C =  100.0
gamma =  0.01
Accuracy:  0.945


Model 6:
C =  100.0
gamma =  0.1
Accuracy:  0.9195


Model 7:
C =  10000.0
gamma =  0.001
Accuracy:  0.9450000000000001


Model 8:
C =  10000.0
gamma =  0.01
Accuracy:  0.9564999999999999


Model 9:
C =  10000.0
gamma =  0.1
Accuracy:  0.9190000000000002



Optimal parameters for the model:
C =  10000.0
gamma =  0.01
Accuracy:  0.9564999999999999


As we can see again, for constant values of gamma, different values of C affect just slightly the accuracy of the model. However, gamma seems to play an important role with the model performing clearly better when gamma=0.01. More specifically, for that value of gamma, the model performs better when we have a large value of C, too. This can be explained by the fact that for that value of gamma, the transformed data become as linearly separable as they can be in the high-dimensional space and as a result it is better to penalize misclassified samples more.

In [ ]:
# Retrain best Gaussian kernel SVM
svm_model = SVM(kernel=gaussian_kernel, C=best_C, gamma=best_gamma ,disp=True)

# Train model
svm_model.fit(x_train_mini,y_train_mini)

# Evaluate model on test set
y_pred = svm_model.predict(x_test_pca)

print('Test set accuracy: ', accuracy_score(y_test,y_pred))
print(classification_report(y_test, y_pred))

Number of samples: 2000
Number of features: 87
Gram matrix elapsed time: 60.61814045906067s

QP solver elapsed time: 15.361772775650024s

Testing elapsed time: 91.80328059196472s

Test set accuracy:  0.9603
              precision    recall  f1-score   support

        -1.0       0.96      0.96      0.96      4926
         1.0       0.96      0.96      0.96      5074

    accuracy                           0.96     10000
   macro avg       0.96      0.96      0.96     10000
weighted avg       0.96      0.96      0.96     10000



Comparing execution times with the RBF SVM model we had in Part 1, we can see that different values of gamma don't really seem to affect the time needed to calculate the Gram matrix. However, it took a bit more time here to solve the optimization problem. \
As for the performance of the model, we can see that its accuracy is the highest we have achieved compared with the rest of the models we have tried out so far.

# **Part 3:** SVM with SMO implementation

As we mentioned earlier, one of the main problems of trying to solve the optimization problem with a QP solver is that we need to calculate the Gram matrix that occurs from the training set samples. However, this requires a lot of memory, especially when the training set is large.\
A way to tackle this is by breaking the original QP problem into smaller QP subproblems that can be solved more easily and the solution of each one of them contributes to the solution of the original problem. \
An algorithm that is based on this logic is Sequential Minimal Optimization (SMO). SMO tries to optimize only two variables at a time. The advantage of this method is that we can calculate the optimal values of these variables analytically so there is no need to use a QP optimizer.

In [ ]:
class SVM_SMO():

    def __init__(self, max_iter=5000, kernel_type='linear', C=1.0, epsilon=0.001):
        self.kernels = {
            'linear' : self.kernel_linear,
            'quadratic' : self.kernel_quadratic
        }
        self.max_iter = max_iter
        self.kernel_type = kernel_type
        self.C = C
        self.epsilon = epsilon
        
    def objective_function(self, a, X, y):
        # Find  current suport vectors
        sv_indx = a > 1e-3
        y_sv = y[sv_indx]
        x_sv = X[sv_indx,:]
        a_sv = a[sv_indx]

        if self.kernel_type == 'linear':
            result = np.sum(a_sv)-0.5*np.sum((y_sv[:,None]*y_sv[None,:])* \
                            self.kernel_linear(x_sv,x_sv)* \
                            (a_sv[:,None]*a_sv[None,:]))
        else:
            result = np.sum(a_sv)-0.5*np.sum((y_sv[:,None]*y_sv[None,:])* \
                            self.kernel_quadratic(x_sv,x_sv)* \
                            (a_sv[:,None]*a_sv[None,:])) 
        return result
    
    def fit(self, X, y, disp=False):

        # Initialization
        n, d = X.shape[0], X.shape[1]
        alpha = np.zeros((n))
        kernel = self.kernels[self.kernel_type]
        count = 0
        while True:
            count += 1
            alpha_prev = np.copy(alpha)
            for j in range(0, n):
                # Choose two variables to optimize
                i = self.get_rnd_int(0, n-1, j) # Get random int i~=j
                x_i, x_j, y_i, y_j = X[i,:], X[j,:], y[i], y[j]

                k_ij = kernel(x_i, x_i) + kernel(x_j, x_j) - 2 * kernel(x_i, x_j)
                if k_ij == 0:
                    continue
                alpha_prime_j, alpha_prime_i = alpha[j], alpha[i]

                # Compute bounds for the optimized parameters
                (L, H) = self.compute_L_H(self.C, alpha_prime_j, alpha_prime_i, y_j, y_i)

                # Compute model parameters
                self.w = self.calc_w(alpha, y, X)
                self.b = self.calc_b(X, y, self.w)

                # Compute E_i, E_j
                E_i = self.E(x_i, y_i, self.w, self.b)
                E_j = self.E(x_j, y_j, self.w, self.b)

                # Set new alpha values
                alpha[j] = alpha_prime_j + float(y_j * (E_i - E_j))/k_ij
                alpha[j] = max(alpha[j], L)
                alpha[j] = min(alpha[j], H)

                alpha[i] = alpha_prime_i + y_i*y_j * (alpha_prime_j - alpha[j])
            
            if disp is True:
                obj = self.objective_function(alpha,X,y)
                print('Iteration: {}, obective function: {}'.format(count,obj))
            
            # Check convergence
            diff = np.linalg.norm(alpha - alpha_prev)
            if diff < self.epsilon:
                break

            if count >= self.max_iter:
                print("Iteration number exceeded the max of %d iterations" % (self.max_iter))
                return
            
        # Compute final model parameters
        self.b = self.calc_b(X, y, self.w)
        if self.kernel_type == 'linear':
            self.w = self.calc_w(alpha, y, X)
        # Get support vectors
        alpha_idx = np.where(alpha > 0)[0]
        support_vectors = X[alpha_idx, :]
        return support_vectors, count
    
    def calc_b(self, X, y, w):
        b_tmp = y - np.dot(w.T, X.T)
        return np.mean(b_tmp)
    
    def calc_w(self, alpha, y, X):
        return np.dot(X.T, np.multiply(alpha,y))
    
    # Prediction (used in the training step)
    def h(self, X, w, b):
        return np.sign(np.dot(w.T, X.T) + b).astype(int)
    
    # Prediction (used in testing)
    def predict_test(self,x_test):
        return np.sign(np.dot(self.w.T, x_test.T) + self.b).astype(int)
    
    # Prediction error
    def E(self, x_k, y_k, w, b):
        return self.h(x_k, w, b) - y_k
    
    # Computes bounds within which the optimized variables lie
    def compute_L_H(self, C, alpha_prime_j, alpha_prime_i, y_j, y_i):
        if(y_i != y_j):
            return (max(0, alpha_prime_j - alpha_prime_i), min(C, C - alpha_prime_i + alpha_prime_j))
        else:
            return (max(0, alpha_prime_i + alpha_prime_j - C), min(C, alpha_prime_i + alpha_prime_j))
        
    def get_rnd_int(self, a,b,z):
        i = z
        cnt=0
        while i == z and cnt<1000:
            i = rnd.randint(a,b)
            cnt=cnt+1
        return i
    
    # Define kernels
    def kernel_linear(self, x1, x2):
        return np.dot(x1, x2.T)
    def kernel_quadratic(self, x1, x2):
        return (np.dot(x1, x2.T) ** 2)

Optimally, we have to apply grid-search and cross-validation as previously in order to find the best values for C and epsilon. Epsilon works as a tolerance bound: when the norm of alphas (the variables which we optimize) changes less than epsilon we stop training. 

In [ ]:
eps = 1e-3
Ce = 10

# Define a linear SVM with SMO implementation
svm_smo = SVM_SMO(C=Ce, epsilon=eps)

### Train the model with 5000 samples and evaluate it on 1000 samples

In [ ]:
# Create a subset of the training set with 5000 samples
rand_indx = np.random.randint(0,len(x_train_pca),5000)
x_train_mini = x_train_pca[rand_indx,:]
y_train_mini = y_train[rand_indx]

# Train model
start = time.time()
svm_smo.fit(x_train_mini,y_train_mini)
end = time.time()
print("Training time: {}s\n".format(end-start))

# Create a subset of test set with 1000 samples
rand_indx = np.random.randint(0,len(x_test_pca),1000)
x_test_mini = x_test_pca[rand_indx,:]
y_test_mini = y_test[rand_indx]

# Evaluate model
start = time.time()
y_pred_test = svm_smo.predict_test(x_test_mini)
end = time.time()
print("Testing time: {}s\n".format(end-start))

y_pred_train = svm_smo.predict_test(x_train_mini)

print('Training set accuracy: ', accuracy_score(y_train_mini,y_pred_train))
print('Test set accuracy: ', accuracy_score(y_test_mini,y_pred_test))

print(classification_report(y_test_mini, y_pred_test))

Training time: 3312.116627216339s

Testing time: 0.00036144256591796875s

Training set accuracy:  0.891
Test set accuracy:  0.856
              precision    recall  f1-score   support

        -1.0       0.88      0.84      0.86       540
         1.0       0.83      0.87      0.85       460

    accuracy                           0.86      1000
   macro avg       0.86      0.86      0.86      1000
weighted avg       0.86      0.86      0.86      1000



It is clear that the time needed to train an SVM with SMO has increased dramatically. However, this is a necessary compromise we have to make when we cannot solve the problem with an optimized QP solver due to lack of memory. It is worth noticing that inference is extremely quick due to the fact that we have already calculated w and b in the training process so only a dot product needs to be calculated now.\
Regarding the performance of the model, we can see that it's just slightly worse than that of the linear SVM we had in Part 1 (could be because the training set was smaller here).

### Train the model on 20000 samples and evaluate it on 10000 samples

In [ ]:
rand_indx = np.random.randint(0,len(x_train_pca),20000)
x_train_mini = x_train_pca[rand_indx,:]
y_train_mini = y_train[rand_indx]

start = time.time()
svm_smo.fit(x_train_mini,y_train_mini)
end = time.time()
print("Training time: {}s\n".format(end-start))

start = time.time()
y_pred_test = svm_smo.predict_test(x_test_pca)
end = time.time()
print("Testing time: {}s\n".format(end-start))

y_pred_train = svm_smo.predict_test(x_train_pca)

print('Training set accuracy: ', accuracy_score(y_train,y_pred_train))
print('Test set accuracy: ', accuracy_score(y_test,y_pred_test))

print(classification_report(y_test, y_pred_test))

Unfortunately, I was unable to train the SVM with such a big training set because of the extremely long time needed for the training to complete (it was taking more than 10 hours and Google Colab timed-out). However, my prediction is that if it was executed we could expect slightly better performance than the previous case due to the fact that the SVM would have seen more samples in the training process thus making it able to generalize better.